In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

StatementMeta(, 8fec420e-c573-4a35-a766-74674b22717d, 3, Finished, Available, Finished)

###### **<mark>Reading data from bronze sales into dataframe</mark>**

In [2]:
%%sql
CREATE TABLE if not EXISTS Ecommerce_project.gold_customer
(
Customer_ID string,
Customer_Name string,
Segment string,
City string,
State string,
Country string,
Region string,
Created_TS timestamp,
Modified_TS timestamp
)
USING DELTA

StatementMeta(, 8fec420e-c573-4a35-a766-74674b22717d, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
Max_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from Ecommerce_project.gold_customer").first()[0]

StatementMeta(, 8fec420e-c573-4a35-a766-74674b22717d, 5, Finished, Available, Finished)

###### **<mark>Creating dataframe with required columns for gold Custimer table</mark>**

In [4]:
df_bronze = spark.read.table("Ecommerce_project.bronze_sales")
df_customer = df_bronze.selectExpr("Customer_ID","Customer_Name","Segment",\
                        "City","State","Country","Region")\
                        .where(col("Modified_TS")>Max_Date) \
                        .dropDuplicates()


StatementMeta(, 8fec420e-c573-4a35-a766-74674b22717d, 6, Finished, Available, Finished)

###### **<mark>Inserting data from Customer_df to dimension table i.e. Gold_customer</mark>**

In [5]:
df_customer.createOrReplaceTempView('ViewCustomer')

StatementMeta(, 8fec420e-c573-4a35-a766-74674b22717d, 7, Finished, Available, Finished)

In [6]:
%%sql
MERGE INTO Ecommerce_project.gold_customer as gc
USING ViewCustomer as vc
on gc.Customer_ID = vc.Customer_ID

WHEN MATCHED THEN
UPDATE set 
gc.Customer_Name = vc.Customer_Name,
gc.Segment = vc.Segment,
gc.City = vc.City,
gc.State = vc.State,
gc.Country = vc.Country,
gc.Region = vc.Region,
gc.Modified_TS = CURRENT_TIMESTAMP

WHEN NOT MATCHED THEN
INSERT
(
gc.Customer_ID,
gc.Customer_Name,
gc.Segment,
gc.City ,
gc.State,
gc.Country,
gc.Region ,
gc.Created_TS,
gc.Modified_TS
)
VALUES
(
vc.Customer_ID,
vc.Customer_Name,
vc.Segment,
vc.City ,
vc.State,
vc.Country,
vc.Region ,
CURRENT_TIMESTAMP(),
CURRENT_TIMESTAMP()
)

StatementMeta(, 8fec420e-c573-4a35-a766-74674b22717d, 8, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>